# Question to be answered:

- Is the accuracy of model/human significantly better? In both force and mass questions?
- Is the distribution of responses significantly different?

In [1]:
import isaac.constants
isaac.constants.TQDM_DISABLE = True

from torch import nn
from isaac.utils import get_cuda_device_if_available
import joblib

from isaac.dataset import read_dataset, prepare_dataset
from isaac.models import MultiBranchModel
from isaac.constants import BASIC_TRAINING_COLS, MASS_CLASS_COLS, FORCE_CLASS_COLS
from isaac.evaluation import evaluate_saved_model
from isaac.statistical_tests import z_test

import torch
import glob
from torch.autograd import Variable
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
device = get_cuda_device_if_available()
print(device)

cuda:0


In [3]:
def get_question_accuracy_for_group_of_models(question_type):
    normalise_data = True
    scaler_path = "scalers/passive_dual_scaler.sk"
    network_dims = (len(BASIC_TRAINING_COLS), 25, 3, 0.5)
    dataset_path = "data/passive_trials_exp1.h5"
    class_columns = [list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)]
    multiclass = True
    seq_end = 1800
    step_size = 3
    
    models = sorted(glob.glob("models/best_"+question_type+"_model_seed_*.pt"))

    group_accuracy = []
    group_predictions = []

    for model_path in tqdm(models):
        accuracies, predicted = evaluate_saved_model(model_path, network_dims, dataset_path, 
                                                     training_columns=BASIC_TRAINING_COLS, class_columns=class_columns, 
                                                     step_size=step_size, seq_end=seq_end, scaler_path=scaler_path,
                                                     arch=MultiBranchModel, multiclass=multiclass)

        if question_type == "mass":
            accuracy = accuracies[0]
            predicted = predicted[:, 0]
        else:
            accuracy = accuracies[1]
            predicted = predicted[:, 1]

        group_accuracy.append(accuracy)
        group_predictions.append(predicted.numpy())

    return group_accuracy, group_predictions

def get_participant_accuracy(passive_responses, answer_column, question_type_answer):
    return [(df[answer_column] == df[question_type_answer]).sum() / len(df) 
            for participant_id, df in passive_responses.groupby("participant")]


def get_participant_accuracy_filtering_by_answer(passive_responses, answer_column, question_type_answer, filter_by_class):
    
    passive_responses = passive_responses.copy().query(question_type_answer+" == "+filter_by_class)
    
    return [(df[answer_column] == df[question_type_answer]).sum() / len(df) 
            for participant_id, df in passive_responses.groupby("participant")]

# T-test for MASS questions

## Load model's predictions

In [4]:
question_type = "mass"
group_mass_acc, group_mass_prediction = get_question_accuracy_for_group_of_models(question_type)
     
question_type = "force"
group_force_acc, group_force_prediction = get_question_accuracy_for_group_of_models(question_type)

 20%|██        | 1/5 [00:03<00:12,  3.05s/it]

Model's accuracy on test set: [54.16666667 56.94444444]


 40%|████      | 2/5 [00:05<00:08,  2.89s/it]

Model's accuracy on test set: [56.01851852 52.77777778]


 60%|██████    | 3/5 [00:07<00:05,  2.74s/it]

...

Model's accuracy on test set: [53.7037037  59.25925926]


 80%|████████  | 4/5 [00:09<00:02,  2.37s/it]

Model's accuracy on test set: [53.24074074 62.03703704]


100%|██████████| 5/5 [00:12<00:00,  2.42s/it]

Model's accuracy on test set: [54.62962963 62.5       ]


In [5]:
np.mean(group_mass_acc), np.mean(group_force_acc)

(55.462962962962955, 59.62962962962963)

In [6]:
from scipy.stats import mode

group_mass_prediction = np.array(group_mass_prediction)
group_force_prediction = np.array(group_force_prediction)

majority_mass_predictions, _ = mode(group_mass_prediction, axis=0)
majority_force_predictions, _ = mode(group_force_prediction, axis=0)

In [7]:
ensemble_mass_answers = [MASS_CLASS_COLS[i] for i in majority_mass_predictions[0]]
ensemble_force_answers = [FORCE_CLASS_COLS[i] for i in majority_force_predictions[0]]

## Load human results

In [8]:
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()

/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Warning messages:

  warnings.warn(x, RRuntimeWarning)
/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 1: 
  warnings.warn(x, RRuntimeWarning)
/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: In value[[3L]](cond) :
  warnings.warn(x, RRuntimeWarning)
/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 
 
  warnings.warn(x, RRuntimeWarning)
/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  "getThreads" not available for .C() for package "RevoUtilsMath"

  warnings.warn(x, RRuntimeWarning)
/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2: 
  warnings.warn(x, RRuntimeWa

In [9]:
rdata_path = "for_hector_small/data/exp1.rdata"
r['load'](rdata_path)

is_passive = (r['dfc.l'].condition == '0') & (r['dfc.l'].practice == 0.) & (r['dfc.l'].exclude == 0)
responses = r['dfc.l'][["participant", "mass", "relationship", "trueMass", "trueRelationship", 'post_ent', 'post_ent_rel.rtheta', 'post_ent_mass.rtheta', 'corMass', 'corRel']]
passive_responses = responses[is_passive]

/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [10]:
passive_responses["model_mass"] = ensemble_mass_answers
passive_responses["model_relationship"] = ensemble_force_answers

/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
human_mass_accuracy_list = get_participant_accuracy(passive_responses, "mass", "trueMass")
human_force_accuracy_list = get_participant_accuracy(passive_responses, "relationship", "trueRelationship")

model_mass_accuracy_list = get_participant_accuracy(passive_responses, "model_mass", "trueMass")
model_force_accuracy_list = get_participant_accuracy(passive_responses, "model_relationship", "trueRelationship")

In [12]:
print(np.mean(human_mass_accuracy_list), "±", np.std(human_mass_accuracy_list))
print(np.mean(model_mass_accuracy_list), "±", np.std(model_mass_accuracy_list))

0.4537037037037037 ± 0.1922272175187657
0.5787037037037037 ± 0.16348558873157015


In [13]:
print(np.mean(model_force_accuracy_list), "±", np.std(model_force_accuracy_list))
print(np.mean(human_force_accuracy_list), "±", np.std(human_force_accuracy_list))

0.6574074074074073 ± 0.16640926625566138
0.611111111111111 ± 0.2151657414559676


# Perform t-test on overall accuracy

In [14]:
# Null hypothesis (opposite of what we want to prove): accuracies are not significantly different
# Alternative hypothesis: accuracies are significantly different

In [15]:
from scipy.stats import ttest_ind, ttest_rel, f_oneway

#### Is the model significantly better answering mass questions than humans?

In [16]:
ttest_rel(human_mass_accuracy_list, model_mass_accuracy_list)

Ttest_relResult(statistic=-2.3305550584499115, pvalue=0.028907500943665062)

#### Is the model significantly better answering force questions than humans?

In [17]:
ttest_rel(human_force_accuracy_list, model_force_accuracy_list)

Ttest_relResult(statistic=-0.7382572022557348, pvalue=0.4678248501407801)

#### Is the model significantly better answering force questions than mass questions?

In [18]:
ttest_ind(model_mass_accuracy_list, model_force_accuracy_list)

Ttest_indResult(statistic=-1.6180110712548972, pvalue=0.11249773981293777)

# Perform z-test on overall accuracy

In [19]:
# Null hypothesis (opposite of what we want to prove): accuracies are not significantly different
# Alternative hypothesis: classifier 2 is significantly more accurate than the first

z_zero_point_five = 1.645

Z = z_test(passive_responses["trueMass"], passive_responses["mass"], passive_responses["model_mass"])

print(Z)
if Z < -z_zero_point_five:
    print("Classifier two significantly better than classifier one.")
else:
    print("Classifier two not significantly better.")

Z = z_test(passive_responses["trueRelationship"], passive_responses["relationship"], passive_responses["model_relationship"])

print(Z)
if Z < -z_zero_point_five:
    print("Classifier two significantly better than classifier one.")
else:
    print("Classifier two not significantly better than classifier one.")

# Are correct guesses / errors correlated between humans and model?

In [20]:
from scipy.stats import chisquare

In [21]:
passive_responses["model_mass_correct_guesses"] = (passive_responses["model_mass"] == passive_responses["trueMass"])
passive_responses["model_force_correct_guesses"] = (passive_responses["model_relationship"] == passive_responses["trueRelationship"])

/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
mass_coincidence = get_participant_accuracy(passive_responses, "mass", "model_mass")
force_coincidence = get_participant_accuracy(passive_responses, "relationship", "model_relationship")

print(np.mean(mass_coincidence), np.std(mass_coincidence))
print(np.mean(force_coincidence), np.std(force_coincidence))

0.4537037037037037 0.15354744399793518
0.46759259259259256 0.15375668430940384


In [23]:
def test_for_independence(first_answers, second_answers):
    both_correct = (first_answers & second_answers).sum()
    both_wrong = (~first_answers & ~second_answers).sum()
    first_correct_second_wrong = (~first_answers & second_answers).sum()
    first_wrong_second_correct = (first_answers & ~second_answers).sum()
    
    matrix = [[both_correct, first_correct_second_wrong], [first_wrong_second_correct, both_wrong]]
    chisquare_results = chisquare(matrix, axis=None)
    
    return matrix, chisquare_results

In [24]:
test_for_independence(passive_responses["corRel"], passive_responses["model_force_correct_guesses"])

([[83, 59], [49, 25]],
 Power_divergenceResult(statistic=32.074074074074076, pvalue=5.0486794044425e-07))

In [25]:
test_for_independence(passive_responses["corMass"], passive_responses["model_mass_correct_guesses"])

([[59, 66], [39, 52]],
 Power_divergenceResult(statistic=7.37037037037037, pvalue=0.06098442930559373))

In [26]:
def test_for_coincidence(first_answers, second_answers):
    
    matrix = []
    all_classes = first_answers.unique()
    
    for class_name in all_classes:
        this_class_coincidences = []
        for second_class_name in all_classes:
            n_coincidences = ((first_answers == class_name) & (second_answers == second_class_name)).sum()
            this_class_coincidences.append(n_coincidences)
            
        matrix.append(this_class_coincidences)
    
    chisquare_results = chisquare(matrix, axis=None)
    
    return matrix, chisquare_results, all_classes

In [27]:
test_for_coincidence(passive_responses["mass"], passive_responses["model_mass"])

([[17, 19, 33], [18, 39, 15], [12, 21, 42]],
 Power_divergenceResult(statistic=40.58333333333333, pvalue=2.493864075123097e-06),
 array(['same', 'B', 'A'], dtype=object))

In [28]:
test_for_coincidence(passive_responses["relationship"], passive_responses["model_relationship"])

([[24, 29, 9], [8, 49, 38], [3, 28, 28]],
 Power_divergenceResult(statistic=75.0, pvalue=4.932659739935825e-13),
 array(['attract', 'none', 'repel'], dtype=object))

# Are correct guesses / errors correlated to informativeness?

### Mass responses

In [29]:
not_na_passive_responses = passive_responses[passive_responses["post_ent"].notna()]

post_mass_correct_guesses = not_na_passive_responses.query("model_mass_correct_guesses")["post_ent_mass.rtheta"]
post_mass_wrong_guesses = not_na_passive_responses.query("not model_mass_correct_guesses")["post_ent_mass.rtheta"]

f_oneway(post_mass_correct_guesses, post_mass_wrong_guesses)

F_onewayResult(statistic=1.5323649072780126, pvalue=0.21757124338259207)

In [30]:
post_mass_correct_guesses = not_na_passive_responses.query("corMass == 1")["post_ent_mass.rtheta"]
post_mass_wrong_guesses = not_na_passive_responses.query("corMass == 0")["post_ent_mass.rtheta"]

f_oneway(post_mass_correct_guesses, post_mass_wrong_guesses)

F_onewayResult(statistic=0.03857733827167141, pvalue=0.8445376237361524)

### Force responses

In [31]:
post_force_correct_guesses = not_na_passive_responses[not_na_passive_responses.model_force_correct_guesses]['post_ent_rel.rtheta']
post_force_wrong_guesses = not_na_passive_responses[~not_na_passive_responses.model_force_correct_guesses]['post_ent_rel.rtheta']

f_oneway(post_force_correct_guesses, post_force_wrong_guesses)

F_onewayResult(statistic=3.3989831343693866, pvalue=0.06708642765414946)

In [32]:
post_force_correct_guesses = not_na_passive_responses[(not_na_passive_responses.corRel).astype(bool)]['post_ent_rel.rtheta']
post_force_wrong_guesses = not_na_passive_responses[~(not_na_passive_responses.corRel).astype(bool)]['post_ent_rel.rtheta']

f_oneway(post_force_correct_guesses, post_force_wrong_guesses)

F_onewayResult(statistic=5.982515021800705, pvalue=0.015531865919044263)

#### Same statistic calculated a different way

import statsmodels.api as sm
from statsmodels.formula.api import ols

df_copy = not_na_passive_responses.copy()
df_copy = df_copy.rename({'post_ent_rel.rtheta': 'post_ent_rel_rtheta',
                          'post_ent_mass.rtheta': 'post_ent_mass_rtheta'}, axis=1)

moore_lm = ols('post_ent_mass_rtheta ~ corMass', data=df_copy).fit()
table = sm.stats.anova_lm(moore_lm, typ=2)
print(table)

moore_lm = ols('post_ent_rel_rtheta ~ corRel', data=df_copy).fit()
table = sm.stats.anova_lm(moore_lm, typ=2)
print(table)

# Is the model better at predicting any force class?

In [33]:
repel_accuracy_list = get_participant_accuracy_filtering_by_answer(passive_responses, "model_relationship", 
                                                                   "trueRelationship", "'repel'")

none_accuracy_list = get_participant_accuracy_filtering_by_answer(passive_responses, "model_relationship", 
                                                                   "trueRelationship", "'none'")

attract_accuracy_list = get_participant_accuracy_filtering_by_answer(passive_responses, "model_relationship", 
                                                                   "trueRelationship", "'attract'")

In [34]:
print(np.mean(repel_accuracy_list), np.std(repel_accuracy_list))
print(np.mean(none_accuracy_list), np.std(none_accuracy_list))
print(np.mean(attract_accuracy_list), np.std(attract_accuracy_list))

print()

print(ttest_ind(repel_accuracy_list, none_accuracy_list))
print(ttest_ind(repel_accuracy_list, attract_accuracy_list))
print(ttest_ind(attract_accuracy_list, none_accuracy_list))

0.625 0.2602082499332666
0.6875 0.2724311839712921
0.6458333333333334 0.3673998351780916

Ttest_indResult(statistic=-0.795630267734819, pvalue=0.4303336272798818)
Ttest_indResult(statistic=-0.22192461632704463, pvalue=0.8253548952953591)
Ttest_indResult(statistic=-0.4368882801067173, pvalue=0.664236448528826)


# Is the model better at predicting any mass class?

In [35]:
a_accuracy_list = get_participant_accuracy_filtering_by_answer(passive_responses, "model_mass", 
                                                                   "trueMass", "'A'")

same_accuracy_list = get_participant_accuracy_filtering_by_answer(passive_responses, "model_mass", 
                                                                   "trueMass", "'same'")

b_accuracy_list = get_participant_accuracy_filtering_by_answer(passive_responses, "model_mass", 
                                                                   "trueMass", "'B'")

In [36]:
print(np.mean(a_accuracy_list), np.std(a_accuracy_list))
print(np.mean(same_accuracy_list), np.std(same_accuracy_list))
print(np.mean(b_accuracy_list), np.std(b_accuracy_list))

print()

print(ttest_ind(a_accuracy_list, same_accuracy_list))
print(ttest_ind(a_accuracy_list, b_accuracy_list))
print(ttest_ind(b_accuracy_list, same_accuracy_list))

0.7222222222222222 0.2664350846284844
0.375 0.24176243313235013
0.6388888888888888 0.23405971592156552

Ttest_indResult(statistic=4.6285255624513155, pvalue=3.0219150546836055e-05)
Ttest_indResult(statistic=1.126915547104906, pvalue=0.26562258203569544)
Ttest_indResult(statistic=3.76095786849388, pvalue=0.00047715292903339075)


# Is the model better at predicting any force class than humans?

In [37]:
human_repel_accuracy_list = get_participant_accuracy_filtering_by_answer(passive_responses, "relationship", 
                                                                         "trueRelationship", "'repel'")

human_none_accuracy_list = get_participant_accuracy_filtering_by_answer(passive_responses, "relationship", 
                                                                        "trueRelationship", "'none'")

human_attract_accuracy_list = get_participant_accuracy_filtering_by_answer(passive_responses, "relationship", 
                                                                   "trueRelationship", "'attract'")

print(np.mean(human_repel_accuracy_list), np.std(human_repel_accuracy_list))
print(np.mean(human_none_accuracy_list), np.std(human_none_accuracy_list))
print(np.mean(human_attract_accuracy_list), np.std(human_attract_accuracy_list))

0.5277777777777778 0.3458305443885759
0.6041666666666666 0.25937290829143195
0.75 0.3227486121839514


In [38]:
print(ttest_rel(repel_accuracy_list, human_repel_accuracy_list))
print(ttest_rel(none_accuracy_list, human_none_accuracy_list))
print(ttest_rel(attract_accuracy_list, human_attract_accuracy_list))

Ttest_relResult(statistic=1.0707428896364521, pvalue=0.29539313735467326)
Ttest_relResult(statistic=0.9405399431259601, pvalue=0.3567147403555685)
Ttest_relResult(statistic=-0.9607046637980575, pvalue=0.34669239711665734)


# Is the model better at predicting any mass class than humans?

In [39]:
human_a_accuracy_list = get_participant_accuracy_filtering_by_answer(passive_responses, "mass", 
                                                                     "trueMass", "'A'")

human_same_accuracy_list = get_participant_accuracy_filtering_by_answer(passive_responses, "mass", 
                                                                        "trueMass", "'same'")

human_b_accuracy_list = get_participant_accuracy_filtering_by_answer(passive_responses, "mass", 
                                                                     "trueMass", "'B'")

print(np.mean(human_a_accuracy_list), np.std(human_a_accuracy_list))
print(np.mean(human_same_accuracy_list), np.std(human_same_accuracy_list))
print(np.mean(human_b_accuracy_list), np.std(human_b_accuracy_list))

print()

0.5 0.2545875386086578
0.375 0.3236438943814179
0.4861111111111111 0.33304385578560547



In [40]:
print(ttest_rel(a_accuracy_list, human_a_accuracy_list))
print(ttest_rel(same_accuracy_list, human_same_accuracy_list))
print(ttest_rel(b_accuracy_list, human_b_accuracy_list))

Ttest_relResult(statistic=2.5634797778466227, pvalue=0.017370026723767275)
Ttest_relResult(statistic=-1.118356635373715e-16, pvalue=0.9999999999999999)
Ttest_relResult(statistic=1.9048418372748412, pvalue=0.06938047123922489)
